# ДОМАШНЕЕ ЗАДАНИЕ 4. Рекомендательные системы и Spark MLlib

### Содержание

- [Задание 1. Анализ датасета](#Задание-1.-Анализ-датасета)
- [Задание 2. Коллаборативная фильтрация](#Задание-2.-Коллаборативная-фильтрация)
- [Задание 3. Факторизация матрицы](#Задание-3.-Факторизация-матрицы)

Набор данных:

- [Небольшой набор данных](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip)
- [Полный набор данных](http://files.grouplens.org/datasets/movielens/ml-latest.zip)

⚠️ **Замечания**:
1. Сначала выполните указанные ниже задачи для небольшого набора данных, потом для полного
2. Использовать `Dataframe API`. Исключением может быть задача 2


## **Задание 1**. Анализ датасета

- **Вариант 1.** Animation, Romance, Documentary
- **Вариант 2.** Drama, Comedy, Musical
- **Вариант 3.** Thriller, Sci-Fi, Adventure

⚠️ **Замечание:** Один фильм может принадлежать разным жанрам

1. Выведите данные, сопоставляющие жанры и количество фильмов
2. Выведите первые 10 фильмов с наибольшим количеством рейтингов для каждого жанра в соответствии с вариантом
3. Выведите первые 10 фильмов с наименьшим количеством рейтингов (но больше 10) для каждого жанра в соответствии с вариантом
4. Выведите первые 10 фильмов с наибольшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом
5. Выведите первые 10 фильмов с наименьшим средним рейтингом при количестве рейтингов больше 10 для каждого жанра в соответствии с вариантом


## **Задание 2**. Коллаборативная фильтрация

- **Вариант 1.** По схожести пользователей
- **Вариант 2.** По схожести объектов

1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите `rmse` для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета `rmse` - test
3. Определите `rmse` для тестового подмножества

## **Задание 3**. Факторизация матрицы

1. Выберите модель `ALS` по минимальному значению `rmse`. Для этого используйте кросс-валидацию `k-folds` c k=4

    Параметры:
    - Количество факторов: `[5, 10, 15]`
    - Регуляризация: `[0.001, 0.01, 0.1, 1, 10]`

    ⚠️ **Замечание:** Если какие-то элементы из тестового/валидационного подмножества не встречались в обучающем, то `rmse` будет `NaN`

2. Сравните результаты рекомендаций посредством коллаборативной фильтрации и факторизации матрицы рейтингов

In [41]:
surname = "Ханенко" #Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [1, 42, 21, 21, 34,  6, 44, 26, 18, 44, 38, 26, 14, 43,  4, 49, 45,
        7, 42, 29,  4,  9, 36, 34, 31, 29,  5, 30,  4, 19, 28, 25, 33]

d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 40 + 1

print("Задача № 1: ", variant % 3 + 1)
print("Задача № 2: ", variant % 2 + 1 )

Задача № 1:  2
Задача № 2:  2


In [42]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

spark = SparkSession.builder.appName("HW4").master("local[*]").getOrCreate()

path = "ml-latest-small/"
movies = spark.read.csv(path + "movies.csv", header=True, inferSchema=True)
ratings = spark.read.csv(path + "ratings.csv", header=True, inferSchema=True)

movies.cache()
ratings.cache()


DataFrame[userId: int, movieId: int, rating: double, timestamp: int]

In [43]:
target = ["Drama", "Comedy", "Musical"]
df_g = movies.withColumn("genre", F.explode(F.split("genres", "\|")))

stats = ratings.groupBy("movieId").agg(
    F.count("rating").alias("cnt"),
    F.avg("rating").alias("avg_r")
)

joined = df_g.join(stats, "movieId")
joined.cache()

DataFrame[movieId: int, title: string, genres: string, genre: string, cnt: bigint, avg_r: double]

In [44]:
print("Количество фильмов по жанрам:")
df_g.filter(F.col("genre").isin(target)).groupBy("genre").count().show()


Количество фильмов по жанрам:
+-------+-----+
|  genre|count|
+-------+-----+
|  Drama| 4361|
|Musical|  334|
| Comedy| 3756|
+-------+-----+



In [45]:
g = "Drama"
print(f"Жанр: {g}")
data = joined.filter(F.col("genre") == g)
print("Топ 10 по количеству оценок:")
data.orderBy(F.col("cnt").desc()).select("title", "cnt").show(10, False)

Жанр: Drama
Топ 10 по количеству оценок:
+--------------------------------+---+
|title                           |cnt|
+--------------------------------+---+
|Forrest Gump (1994)             |329|
|Shawshank Redemption, The (1994)|317|
|Pulp Fiction (1994)             |307|
|Braveheart (1995)               |237|
|Schindler's List (1993)         |220|
|Fight Club (1999)               |218|
|American Beauty (1999)          |204|
|Apollo 13 (1995)                |201|
|Godfather, The (1972)           |192|
|Saving Private Ryan (1998)      |188|
+--------------------------------+---+
only showing top 10 rows



In [46]:
print(f"Жанр: {g}")
print("Топ 10 с наименьшим количеством оценок:")
data.filter("cnt > 10").orderBy(F.col("cnt").asc()).select("title", "cnt").show(10, False)

Жанр: Drama
Топ 10 с наименьшим количеством оценок:
+-----------------------------------------------+---+
|title                                          |cnt|
+-----------------------------------------------+---+
|Across the Universe (2007)                     |11 |
|Messenger: The Story of Joan of Arc, The (1999)|11 |
|Descendants, The (2011)                        |11 |
|Michael Collins (1996)                         |11 |
|Flightplan (2005)                              |11 |
|Solaris (Solyaris) (1972)                      |11 |
|Frailty (2001)                                 |11 |
|He's Just Not That Into You (2009)             |11 |
|39 Steps, The (1935)                           |11 |
|Sicario (2015)                                 |11 |
+-----------------------------------------------+---+
only showing top 10 rows



In [47]:
print(f"Жанр: {g}")
print("Топ 10 по среднему рейтингу:")
data.filter("cnt > 10").orderBy(F.col("avg_r").desc()).select("title", "avg_r").show(10, False)

Жанр: Drama
Топ 10 по среднему рейтингу:
+---------------------------------------------+-----------------+
|title                                        |avg_r            |
+---------------------------------------------+-----------------+
|Secrets & Lies (1996)                        |4.590909090909091|
|Guess Who's Coming to Dinner (1967)          |4.545454545454546|
|Paths of Glory (1957)                        |4.541666666666667|
|Streetcar Named Desire, A (1951)             |4.475            |
|Celebration, The (Festen) (1998)             |4.458333333333333|
|Ran (1985)                                   |4.433333333333334|
|Shawshank Redemption, The (1994)             |4.429022082018927|
|Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)|4.333333333333333|
|Hustler, The (1961)                          |4.333333333333333|
|Double Indemnity (1944)                      |4.323529411764706|
+---------------------------------------------+-----------------+
only showing top 10 rows



In [48]:
print(f"Жанр: {g}")
print("Топ 10 по наименьшему среднему рейтингу:")
data.filter("cnt > 10").orderBy(F.col("avg_r").asc()).select("title", "avg_r").show(10, False)

Жанр: Drama
Топ 10 по наименьшему среднему рейтингу:
+---------------------------------------------------------------------------+------------------+
|title                                                                      |avg_r             |
+---------------------------------------------------------------------------+------------------+
|Karate Kid, Part III, The (1989)                                           |1.75              |
|Beethoven (1992)                                                           |1.7727272727272727|
|Rocky V (1990)                                                             |1.9411764705882353|
|Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)|2.090909090909091 |
|Volcano (1997)                                                             |2.1               |
|Crocodile Dundee in Los Angeles (2001)                                     |2.1923076923076925|
|Clash of the Titans (2010)                                               

In [49]:
g = "Comedy"
print(f"Жанр: {g}")
data = joined.filter(F.col("genre") == g)
print("Топ 10 по количеству оценок:")
data.orderBy(F.col("cnt").desc()).select("title", "cnt").show(10, False)

Жанр: Comedy
Топ 10 по количеству оценок:
+---------------------------------+---+
|title                            |cnt|
+---------------------------------+---+
|Forrest Gump (1994)              |329|
|Pulp Fiction (1994)              |307|
|Toy Story (1995)                 |215|
|Aladdin (1992)                   |183|
|Fargo (1996)                     |181|
|True Lies (1994)                 |178|
|Back to the Future (1985)        |171|
|Shrek (2001)                     |170|
|Men in Black (a.k.a. MIB) (1997) |165|
|Ace Ventura: Pet Detective (1994)|161|
+---------------------------------+---+
only showing top 10 rows



In [50]:
print(f"Жанр: {g}")
print("Топ 10 с наименьшим количеством оценок:")
data.filter("cnt > 10").orderBy(F.col("cnt").asc()).select("title", "cnt").show(10, False)

Жанр: Comedy
Топ 10 с наименьшим количеством оценок:
+----------------------------------------------+---+
|title                                         |cnt|
+----------------------------------------------+---+
|Dumb and Dumberer: When Harry Met Lloyd (2003)|11 |
|Descendants, The (2011)                       |11 |
|Sex and the City (2008)                       |11 |
|Porky's (1982)                                |11 |
|Saving Grace (2000)                           |11 |
|Hotel Transylvania (2012)                     |11 |
|I Now Pronounce You Chuck and Larry (2007)    |11 |
|He's Just Not That Into You (2009)            |11 |
|Spanglish (2004)                              |11 |
|Green Hornet, The (2011)                      |11 |
+----------------------------------------------+---+
only showing top 10 rows



In [51]:
print(f"Жанр: {g}")
print("Топ 10 по среднему рейтингу:")
data.filter("cnt > 10").orderBy(F.col("avg_r").desc()).select("title", "avg_r").show(10, False)

Жанр: Comedy
Топ 10 по среднему рейтингу:
+---------------------------------------------------------------------------+------------------+
|title                                                                      |avg_r             |
+---------------------------------------------------------------------------+------------------+
|His Girl Friday (1940)                                                     |4.392857142857143 |
|It Happened One Night (1934)                                               |4.321428571428571 |
|Philadelphia Story, The (1940)                                             |4.310344827586207 |
|Living in Oblivion (1995)                                                  |4.3076923076923075|
|Harold and Maude (1971)                                                    |4.288461538461538 |
|Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)|4.268041237113402 |
|Creature Comforts (1989)                                                   |4.25    

In [52]:
print(f"Жанр: {g}")
print("Топ 10 по наименьшему среднему рейтингу:")
data.filter("cnt > 10").orderBy(F.col("avg_r").asc()).select("title", "avg_r").show(10, False)

Жанр: Comedy
Топ 10 по наименьшему среднему рейтингу:
+----------------------------------------------+------------------+
|title                                         |avg_r             |
+----------------------------------------------+------------------+
|Problem Child (1990)                          |1.5833333333333333|
|Flintstones in Viva Rock Vegas, The (2000)    |1.625             |
|Stop! Or My Mom Will Shoot (1992)             |1.7727272727272727|
|Beethoven (1992)                              |1.7727272727272727|
|Dungeons & Dragons (2000)                     |1.8333333333333333|
|Sister Act 2: Back in the Habit (1993)        |1.8928571428571428|
|Dumb and Dumberer: When Harry Met Lloyd (2003)|1.9545454545454546|
|Super Mario Bros. (1993)                      |2.0               |
|Honey, I Blew Up the Kid (1992)               |2.05              |
|Teenage Mutant Ninja Turtles III (1993)       |2.0714285714285716|
+----------------------------------------------+--------------

In [53]:
g = "Musical"
print(f"Жанр: {g}")
data = joined.filter(F.col("genre") == g)
print("Топ 10 по количеству оценок:")
data.orderBy(F.col("cnt").desc()).select("title", "cnt").show(10, False)

Жанр: Musical
Топ 10 по количеству оценок:
+-------------------------------------------+---+
|title                                      |cnt|
+-------------------------------------------+---+
|Aladdin (1992)                             |183|
|Lion King, The (1994)                      |172|
|Beauty and the Beast (1991)                |146|
|Willy Wonka & the Chocolate Factory (1971) |119|
|Nightmare Before Christmas, The (1993)     |93 |
|Shrek 2 (2004)                             |92 |
|Wizard of Oz, The (1939)                   |92 |
|Blues Brothers, The (1980)                 |84 |
|Snow White and the Seven Dwarfs (1937)     |77 |
|South Park: Bigger, Longer and Uncut (1999)|76 |
+-------------------------------------------+---+
only showing top 10 rows



In [54]:
print(f"Жанр: {g}")
print("Топ 10 с наименьшим количеством оценок:")
data.filter("cnt > 10").orderBy(F.col("cnt").asc()).select("title", "cnt").show(10, False)

Жанр: Musical
Топ 10 с наименьшим количеством оценок:
+--------------------------------------+---+
|title                                 |cnt|
+--------------------------------------+---+
|Across the Universe (2007)            |11 |
|Doctor Dolittle (1967)                |11 |
|Chitty Chitty Bang Bang (1968)        |11 |
|Oliver & Company (1988)               |11 |
|All Dogs Go to Heaven 2 (1996)        |11 |
|Hedwig and the Angry Inch (2000)      |11 |
|How the Grinch Stole Christmas! (1966)|11 |
|Evita (1996)                          |12 |
|Princess and the Frog, The (2009)     |12 |
|Victor/Victoria (1982)                |12 |
+--------------------------------------+---+
only showing top 10 rows



In [55]:
print(f"Жанр: {g}")
print("Топ 10 по среднему рейтингу:")
data.filter("cnt > 10").orderBy(F.col("avg_r").desc()).select("title", "avg_r").show(10, False)


Жанр: Musical
Топ 10 по среднему рейтингу:
+-------------------------------------+------------------+
|title                                |avg_r             |
+-------------------------------------+------------------+
|Hedwig and the Angry Inch (2000)     |4.181818181818182 |
|King and I, The (1956)               |4.166666666666667 |
|Singin' in the Rain (1952)           |4.074468085106383 |
|Across the Universe (2007)           |4.045454545454546 |
|My Fair Lady (1964)                  |4.042857142857143 |
|Dancer in the Dark (2000)            |3.975             |
|Lion King, The (1994)                |3.941860465116279 |
|Sound of Music, The (1965)           |3.9375            |
|Tangled (2010)                       |3.9166666666666665|
|Dr. Horrible's Sing-Along Blog (2008)|3.9166666666666665|
+-------------------------------------+------------------+
only showing top 10 rows



In [56]:
print(f"Жанр: {g}")
print("Топ 10 по наименьшему среднему рейтингу:")
data.filter("cnt > 10").orderBy(F.col("avg_r").asc()).select("title", "avg_r").show(10, False)


Жанр: Musical
Топ 10 по наименьшему среднему рейтингу:
+--------------------------------------+------------------+
|title                                 |avg_r             |
+--------------------------------------+------------------+
|Grease 2 (1982)                       |2.0789473684210527|
|Popeye (1980)                         |2.5               |
|Blues Brothers 2000 (1998)            |2.7083333333333335|
|Pete's Dragon (1977)                  |2.7666666666666666|
|Mamma Mia! (2008)                     |2.9642857142857144|
|Oliver & Company (1988)               |3.090909090909091 |
|Annie (1982)                          |3.125             |
|Grease (1978)                         |3.139705882352941 |
|Pocahontas (1995)                     |3.1470588235294117|
|Aladdin and the King of Thieves (1996)|3.1538461538461537|
+--------------------------------------+------------------+
only showing top 10 rows



In [57]:
train, test = ratings.randomSplit([0.8, 0.2], seed=42)

gl_avg = train.select(F.avg("rating")).first()[0]
print("Среднее по обучающей выборке:", gl_avg)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
test_base = test.withColumn("prediction", F.lit(gl_avg))
rmse_base = evaluator.evaluate(test_base)
print("RMSE базовой модели:", rmse_base)

item_avgs = train.groupBy("movieId").agg(F.avg("rating").alias("i_avg"))
preds = test.join(item_avgs, "movieId", "left") \
    .withColumn("prediction", F.coalesce(F.col("i_avg"), F.lit(gl_avg)))

rmse_item = evaluator.evaluate(preds)
print("RMSE по схожести объектов:", rmse_item)


Среднее по обучающей выборке: 3.5039340762987417
RMSE базовой модели: 1.050437770716982
RMSE по схожести объектов: 0.9765692832448408


In [58]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")

grid = ParamGridBuilder() \
    .addGrid(als.rank, [5, 10, 15]) \
    .addGrid(als.regParam, [0.001, 0.01, 0.1, 1, 10]) \
    .build()

cv = CrossValidator(estimator=als, estimatorParamMaps=grid, evaluator=evaluator, numFolds=4)

model = cv.fit(train)
best = model.bestModel

print("Лучший rank:", best.rank)
print("Лучший regParam:", best._java_obj.parent().getRegParam())

res_als = best.transform(test)
rmse_als = evaluator.evaluate(res_als)

print("Итоговое сравнение RMSE:")
print("Базовая:", rmse_base)
print("По объектам:", rmse_item)
print("ALS:", rmse_als)


Лучший rank: 5
Лучший regParam: 0.1
Итоговое сравнение RMSE:
Базовая: 1.050437770716982
По объектам: 0.9765692832448408
ALS: 0.8799825504640119
